In [1]:
import pandas as pd

#### Clean up function for population data downloaded from statistics.gov.scot
##### You can only export a single gender so this functions helps to combine the two downloaded files into one

In [2]:
def data_merge(df_male, df_female, year):

    df_m = df_male.copy()
    df_f = df_female.copy()
    
    def data_clean(dataframe, gender):
        df = dataframe.copy()
        df['DZ'] = df.iloc[:,0].str[52:]
        #remove # to uncomment the correct datazone mask
        mask = df['DZ'].str.contains('S0100234[1-7]') #2001 Uist datazones
        #mask = df['DZ'].str.contains('S010090([1][6-9]|[2][0-2])') #2011 Uist datazones
        df = df[mask].iloc[:,6:].assign(Gender=gender)
        return df
    
    df_m = data_clean(df_m, 'Male')
    df_f = data_clean(df_f, 'Female')
    
    df = pd.concat([df_m, df_f])
    df['Year'] = year
    age_cols = df.columns.values[:-3]
    df = (df.melt(id_vars=['DZ', 'Year', 'Gender'], value_vars=age_cols,
                    var_name='Age_Bands', value_name='Population')
            .groupby(['Year', 'Gender', 'Age_Bands'])['Population']
                    .sum()
                    .reset_index(name='Population')
            .replace({'Age_Bands':{'90 And Over':'90+'}}))
    
    return df

In [3]:
#As an example, I've downloaded 2001 and 2009 files; this script will merge them into one CSV

m = pd.read_csv(r'data\open data downloads\2009_male.csv',skiprows=8)
f = pd.read_csv(r'data\open data downloads\2009_female.csv',skiprows=8)
data_merge(m, f, '2009').to_csv(r'data\2009_clean.csv', index=False)